<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project files files from github 


In [ ]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

import libraries


In [5]:
import torch
import os
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import numpy as np

# project classes --------------------------------------------------------------
from dataset.ilcifar100 import ilCIFAR100
from baselines.resnet import resnet32
from info_recorder.info_log import InfoLog
from models_benchmark.benchmark import Benchmark

Initial setup

In [9]:
# Define params

NUM_RUN = 3
SEEDS = [1,2,3]
NUM_EPOCHS = 70
BATCH_SIZE = 128
NUM_CLASS_BATCH = 10
DEVICE = 'cuda'

# Create savings folder
try:
    os.mkdir("savings")
except FileExistsError:
    pass

Define transforms

In [10]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

Prepare dataloders for each train, val test subset of the original dataset

In [ ]:
# dataloaders dict initialization
dataloaders = {run:{batch:{'train':None, 'val':None, 'test':None} for batch in range(NUM_CLASS_BATCH)} for run in range(NUM_RUN)}

# make dataloaders
for run in range(NUM_RUN):
    
    original_training_set = ilCIFAR100(root="cifar", seed=SEEDS[run], transform=train_transform)
    original_test_set = ilCIFAR100(root="cifar", train=False, seed=SEEDS[run], transform=test_transform)
    
    for class_batch in range(NUM_CLASS_BATCH):
        
        # get indices
        train_indices = original_training_set.batches[class_batch]['train']
        val_indices = original_training_set.batches[class_batch]['val']
        test_indices = original_test_set.batches[class_batch]

        # make subsets
        train_set = Subset(dataset=original_training_set, indices=train_indices)
        val_set = Subset(dataset=original_training_set, indices=val_indices)
        test_set = Subset(dataset=original_test_set, indices=test_indices)

        # make dataloaders
        train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
        test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

        # add dataloaders to the dataloaders dict
        dataloaders[run][class_batch]['train'] = train_dataloader
        dataloaders[run][class_batch]['val'] = val_dataloader
        dataloaders[run][class_batch]['test'] = test_dataloader